## Data recovery

In [ ]:
from sqlalchemy.orm import sessionmaker
from models import City, DbInteraction
from sqlalchemy import select
from csv import DictReader

In [ ]:
db = DbInteraction('localhost,5432', 'mex_covid_py','postgres', 'this_isAstrong#password')
Session = sessionmaker(bind=db.engine)
ses = Session()

In [ ]:
cities = {}
daily_city_summaries = []

In [ ]:
result = ses.execute(select(City))
for city in result.scalars().all():
    cities[(city.city_code, city.state_id)] = {'confirmed': 0, 'c_name': city.city_name, 'lat': city.latitude, 'lon': city.longitude}

In [ ]:
def get_indicators(cities, file):
    with open(file, 'r', encoding="ISO-8859-1") as csv_f:
        f = DictReader(csv_f)
        for row in f:
            if int(row['RESULTADO']) == 1:
                city_index = (int(row['MUNICIPIO_RES']),int(row['ENTIDAD_RES']))
                if city_index in cities:
                    cities[city_index]['confirmed'] += 1
    return cities

## Data visualization

In [ ]:
import folium

In [ ]:
cities_confirmed = get_indicators(cities, 'zip/200912COVID19MEXICO.csv')

In [ ]:
m = folium.Map(location=[23.634501, -102.552784], tiles="OpenStreetMap", zoom_start=5)

In [ ]:
# Create the points
for city in cities_confirmed.values():
    if city['confirmed'] > 1000:
        folium.Circle(
            location=[city['lat'], city['lon']],
            popup=f'{city["c_name"].title()} confirmed:{city["confirmed"]}',
            radius=float(city['confirmed']),
            color='crimson',
            fill=True,
            fill_color='crimson'
        ).add_to(m)
# Show map
# m